In [1]:
# Set CWD to correct directory
# Do not run more than once

import os
# wd = "C:/Users/lilba/Documents/COLLEGE/RIT Research/Interception_UXF_Analysis-CalibAssessment/Modules"
wd = os.getcwd()
os.chdir('/'.join(wd.split('/')[:-1])+'/')
print('CWD:'+os.getcwd())

CWD:/Users/gjdpci/Dropbox/Code/Catching - IPD/expansion analysis - UXF 1/Interception_UXF_Analysis


In [2]:
import sys

sys.path.append("Modules/")
sys.path.append("/")


import logging
import pickle
import numpy as np
import pandas as pd


fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from loadData import unpackSession

# sys.path.append("pyFiles/")
# from processData import *

## Import raw subject data (slow) or previously imported data from file (fast)
**Set doNotLoad to False to check if pickle file exists and quickly load it instead of slowly generating new data.**

In [3]:
doNotLoad = False
subNum = 0
sessionDict = unpackSession(subNum, doNotLoad=doNotLoad)

INFO_loadData-unpackSession(): - Processing session: Data/P_201218121321_sub1
INFO_loadData-unpackSession(): - Importing session dict from pickle.


***> 0: P_201218121321_sub1
1: P_201218121321_sub1c
2: P_201218121321_sub1.zip
3: P_201218121321_sub1b
4: P_200917094202
5: P_201219105516


In [4]:
list(sessionDict.keys())

['subID',
 'trialInfo',
 'expConfig',
 'rawExpUnity',
 'rawExpGaze',
 'processedExp',
 'rawCalibUnity',
 'rawCalibGaze',
 'processedCalib',
 'analysisParameters']

In [5]:
sessionDict['trialInfo'].keys()

MultiIndex([(                               'ballFinalPos', 'x'),
            (                               'ballFinalPos', 'y'),
            (                               'ballFinalPos', 'z'),
            (                             'ballInitialPos', 'x'),
            (                             'ballInitialPos', 'y'),
            (                             'ballInitialPos', 'z'),
            (                             'ballInitialVel', 'x'),
            (                             'ballInitialVel', 'y'),
            (                             'ballInitialVel', 'z'),
            (                                'blockNumber',  ''),
            (                   'camera_movement_filename',  ''),
            (                         'contactLocOnPaddle', 'x'),
            (                         'contactLocOnPaddle', 'y'),
            (                         'contactLocOnPaddle', 'z'),
            (                          'contactLocinWorld', 'x'),
          

In [6]:
numTrials = len(sessionDict['trialInfo'])
trials = range(numTrials)
print('Total trials: ', numTrials)

Total trials:  86


In [7]:
aTrialsInfo = sessionDict['trialInfo'].loc[trials[0]]
print('Trial number: {tNum} \nTrial type: {tType}'.format(tNum = int(aTrialsInfo['trialNumber']),
                                                   tType = str(aTrialsInfo['trialType'].values)))

Trial number: 1 
Trial type: ['CalibrationAssessment']


In [8]:
gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])
gbProcessedCalib_trial.groups.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86])

In [9]:
gbCurrGroup = gbProcessedCalib_trial.get_group(trials[0]+1)  # Add 1 to convert from zero-based to one-based index
gbCurrGroup.keys()

MultiIndex([(         'azimuthWidth',     ''),
            (            'base_data',     ''),
            (          'blockNumber',     ''),
            (               'camera', 'R0C0'),
            (               'camera', 'R0C1'),
            (               'camera', 'R0C2'),
            (               'camera', 'R0C3'),
            (               'camera', 'R1C0'),
            (               'camera', 'R1C1'),
            (               'camera', 'R1C2'),
            (               'camera', 'R1C3'),
            (               'camera', 'R2C0'),
            (               'camera', 'R2C1'),
            (               'camera', 'R2C2'),
            (               'camera', 'R2C3'),
            (               'camera', 'R3C0'),
            (               'camera', 'R3C1'),
            (               'camera', 'R3C2'),
            (               'camera', 'R3C3'),
            (            'cameraPos',    'x'),
            (            'cameraPos',    'y'),
            (

# Create graphs for azimuth and elevation, comparing against local target
* Separated by trial number
* Axis range is set at the top

# Calc cyclopean 

sessionDict['processedCalib']['cycEIH']

sessionDict['processedCalib']['cycEIH_el']

sessionDict['processedCalib']['cycEIH_az']

In [10]:
def calcCycEIH(sessionDictIn):
    
    gazeNormal2 = np.nanmean([sessionDictIn['processedCalib']['gaze-normal0'],sessionDictIn['processedCalib']['gaze-normal1']],axis=0)

    sessionDictIn['processedCalib'][('cycEIH','x')] = gazeNormal2[:,0]
    sessionDictIn['processedCalib'][('cycEIH','y')] = gazeNormal2[:,1]
    sessionDictIn['processedCalib'][('cycEIH','z')] = gazeNormal2[:,2]

    # Normalize!
    sessionDictIn['processedCalib']['cycEIH'] = np.array([np.divide(XYZ,np.linalg.norm(XYZ)) for XYZ in sessionDict['processedCalib']['cycEIH'].values],dtype=np.float)
    
    # While we're at it, let's calc spherical coordinates az / el
    sessionDictIn['processedCalib']['cycEIH_el'] = sessionDictIn['processedCalib'].apply(lambda row: np.rad2deg(np.arctan2(row[('cycEIH','y')],row[('cycEIH','z')])),axis=1)
    sessionDictIn['processedCalib']['cycEIH_az'] = sessionDictIn['processedCalib'].apply(lambda row: np.rad2deg(np.arctan2(row[('cycEIH','x')],row[('cycEIH','z')])),axis=1)
    
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIH\']')
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIH_az\']')
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIH_el\']')
    
    return sessionDictIn

sessionDict = calcCycEIH(sessionDict)


/Users/gjdpci/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gjdpci/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
INFO___main__-calcCycEIH(): - Added sessionDict['processedCalib']['cycEIH']
INFO___main__-calcCycEIH(): - Added sessionDict['processedCalib']['cycEIH_az']
INFO___main__-calcCycEIH(): - Added sessionDict['processedCalib']['cycEIH_el']


# Filter data

In [11]:

def filterEIH(sessionDict):

    sgWinSizeSamples = sessionDict['analysisParameters']['sgWinSizeSamples']
    sgPolyorder = sessionDict['analysisParameters']['sgPolyorder']
    medFiltSize = sessionDict['analysisParameters']['medFiltSize']
    interpResS = sessionDict['analysisParameters']['interpResS']
    sgWinSizeSamples = sessionDict['analysisParameters']['sgWinSizeSamples']

    from scipy.signal import savgol_filter

    # FIlter
    proc = sessionDict['processedCalib']

    # get ∆T per frame
    frameDur = np.float(proc['frameTime'].diff().mode()[0])
    
    # Median filter to remove outliers
    proc['cycEIHFilt_az'] = proc['cycEIH_az'].rolling(medFiltSize).median()
    proc['cycEIHFilt_el'] = proc['cycEIH_el'].rolling(medFiltSize).median()

    # Fill in empty values / nans
    proc['cycEIHFilt_az'] = proc['cycEIH_az'].fillna(0)
    proc['cycEIHFilt_el'] = proc['cycEIH_el'].fillna(0)

    # Savitsky-Golay filter for smoothing
    proc['cycEIHFilt_az'] = savgol_filter(proc['cycEIHFilt_az'],
                                                        sgWinSizeSamples,
                                                        sgPolyorder,
                                                        deriv=0,
                                                        delta = frameDur,
                                                        axis=0,
                                                        mode='interp')

    proc['cycGIWFilt_el'] = savgol_filter(proc['cycEIHFilt_el'],
                                                        sgWinSizeSamples,
                                                        sgPolyorder,
                                                        deriv=0,
                                                        delta = frameDur,
                                                        axis=0,
                                                        mode='interp')

    sessionDict['processedCalib'] = proc
    
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIHFilt_az\']')
    logger.info('Added sessionDict[\'processedCalib\'][\'cycEIHFilt_el\']')

    return sessionDict

sessionDict = filterEIH(sessionDict)

INFO___main__-filterEIH(): - Added sessionDict['processedCalib']['cycEIHFilt_az']
INFO___main__-filterEIH(): - Added sessionDict['processedCalib']['cycEIHFilt_el']


# Compare raw and filtered signal for a trial

In [12]:
%matplotlib widget
# %matplotlib inline

In [55]:
import matplotlib.pyplot as plt


tNum = 84 # Trial to plot
yRange = 30 # +/- y degrees
xRange_minMax = [0,1] # [min, max] seconds

gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])
trData = gbProcessedCalib_trial.get_group(tNum)

timestamps = trData['pupilTimestamp'] - trData['pupilTimestamp'].iloc[0]

p = plt.figure(figsize=(15, 10))
plt.style.use('ggplot')

grid = plt.GridSpec(2, 2)#, wspace=0.4, hspace=0.3)
ax = p.add_subplot(grid[:2,:2])
# ax2 = p.add_subplot(grid[:2,:2])

# ax.plot(timestamps,trData['cycEIH_az'],color='c',linewidth=3,alpha = 1.0,linestyle=':',label='raw az')

ax.scatter(timestamps,trData['cycEIH_az'],s=20,color='c',alpha = 1.0,linestyle=':',label='raw az')
ax.plot(timestamps,trData['cycEIHFilt_az'],linestyle='-',color='k',linewidth=2,alpha = 0.2,label='filt az')
ax.scatter(timestamps,trData['cycEIHFilt_az'],s=20,color='k',linewidth=2,alpha = 0.2)

ax.plot(timestamps,trData['cycEIH_el'],color='b',linewidth=3,alpha = 1.0,linestyle=':',label='raw el')
ax.plot(timestamps,trData['cycEIHFilt_el'],linestyle='-',color='g',linewidth=2,alpha = 0.2,label='filt az')
ax.scatter(timestamps,trData['cycEIHFilt_el'],s=20,color='g',linewidth=2,alpha = 0.2)

ax.set_xlabel('time (seconds)')
ax.set_xlim(-xRange_minMax[0], xRange_minMax[1])

ax.set_ylabel('degrees')
ax.set_ylim(-yRange, yRange)


ax.legend(loc= 'upper left')

# fig.figure.savefig('Output/'+str(trialRowIdx+1)+"_azimuth.png")
# plt.pyplot.close(fig.figure)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Record target types (we will need this later)

In [14]:
def findCalibrationTargetType(sessionIn):
    '''
    Input: Session dictionary
    Output:  Session dictionary with new column sessionDict['trialInfo']['targetType']
    '''
    
    gbProcessedCalib_trial = sessionIn['processedCalib'].groupby(['trialNumber'])
    
    targetTypes = []
    for trialRowIdx, trMetaData in sessionIn['trialInfo'].iterrows():

        # This dataframe contains the per-frame processed data associated with this trial
        procDF = gbProcessedCalib_trial.get_group(int(trMetaData['trialNumber']))

        targetType = []


        if ( sum(procDF['isHeadFixed'] == False) ==  len(procDF) ):

            targetType = 'VOR'

        elif( sum(procDF['isHeadFixed'] == True) == len(procDF) ):

            # Count the number of target positions within the local space (head-centered space)
            if( len(procDF['targeLocalPos'].drop_duplicates()) == 1 ):
                # Only one target, so it's a fixation trial
                targetType = 'fixation'
            else:
                # multiple targets, so it's a saccade trial
                targetType = 'fixation+saccade'

        else:
            # The trial has both head fixed and world fixed targets.  
            # We did not plan for that, so let's label it as "unknown."

            targetType = 'unknown'

        print('Trial number: {tNum}, type: {tType}'.format(tNum = int(trMetaData['trialNumber']),
                                                       tType = targetType))
        targetTypes.append(targetType)
        
    
    sessionIn['trialInfo']['targetType'] = targetTypes
    
    logger.info('Added sessionDict[\'trialInfo\'][\'targetType\']')
    
    return sessionDict

sessionDict = findCalibrationTargetType(sessionDict)

Trial number: 1, type: fixation
Trial number: 2, type: fixation
Trial number: 3, type: fixation
Trial number: 4, type: fixation
Trial number: 5, type: fixation
Trial number: 6, type: fixation
Trial number: 7, type: fixation
Trial number: 8, type: fixation
Trial number: 9, type: fixation
Trial number: 10, type: fixation
Trial number: 11, type: fixation
Trial number: 12, type: fixation
Trial number: 13, type: fixation
Trial number: 14, type: fixation
Trial number: 15, type: fixation
Trial number: 16, type: fixation
Trial number: 17, type: fixation
Trial number: 18, type: fixation
Trial number: 19, type: fixation
Trial number: 20, type: fixation
Trial number: 21, type: fixation
Trial number: 22, type: fixation
Trial number: 23, type: fixation
Trial number: 24, type: fixation
Trial number: 25, type: fixation
Trial number: 26, type: fixation
Trial number: 27, type: fixation
Trial number: 28, type: fixation
Trial number: 29, type: fixation
Trial number: 30, type: fixation
Trial number: 31, t

INFO___main__-findCalibrationTargetType(): - Added sessionDict['trialInfo']['targetType']



Trial number: 77, type: fixation
Trial number: 78, type: fixation
Trial number: 79, type: fixation
Trial number: 80, type: fixation
Trial number: 81, type: fixation
Trial number: 82, type: fixation+saccade
Trial number: 83, type: VOR
Trial number: 84, type: VOR
Trial number: 85, type: VOR
Trial number: 86, type: VOR


# Bring target into head-space

In [15]:
tNum = 84 # Trial number

gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])
trData = gbProcessedCalib_trial.get_group(tNum)

rowIn = trData.iloc[200]

# Check to make sure that it is a column vector format.  That is...
# last row is 0,0,0,1
# last column is tX, tY, tZ, 1 (where t denotes translation)
# If not, try the transpose!
headTransform_4x4 = np.reshape(rowIn["camera"].values,[4,4])
headTransform_4x4 = np.array(headTransform_4x4,np.float)

### Take the inverse
The headTransform_4x4 defines the position and orientation of the head space within the world coordinate space.
- Multiplying a vector/point by this matrix will convert world XYZ into their corresponding XYZ in head space.
- That is, it converts from local (head) to global (world) coordinates.
- The inverse does the opposite - it converts from global (world) to local (head) coordinates.
- This is useful, so we can cast the target the target in head coordinates.
- Note that this is actually a costly operation, but I do this because it is instructional rather than efficient.

In [16]:
invHeadTransform_4x4 = np.linalg.inv(headTransform_4x4)

Now, lets multiply the target position in world coordinates by this matrix to convert it into head coordinates.

This requiers converstion into a column vector homogenous coords [X,Y,Z,W=1]

In [17]:
targetWorld_XYZW = np.hstack([rowIn['targetPos'].values,1]).T
targetHead_XYZW = np.dot(invHeadTransform_4x4,targetWorld_XYZW)

"in world: {:.2}, {:.2}, {:.2}, {} ".format(targetWorld_XYZW[0],targetWorld_XYZW[1],targetWorld_XYZW[2], targetWorld_XYZW[3])

'in world: -0.63, 1.0, -1.1, 1 '

In [18]:
"in head: {:.2}, {:.2}, {:.2}, {} ".format(targetHead_XYZW[0],targetHead_XYZW[1],targetHead_XYZW[2],targetHead_XYZW[3])

'in head: 0.02, 0.3, 0.93, 1.0 '

Notice that the format is X Y Z W=1.  We can discard the W=1

# Create graphs for azimuth and elevation, comparing against local target
* Separated by trial number
* Axis range is set at the top

In [46]:
def recalcTargetLocalPosOnVOR(sessionDict):
    
    def recalcTargetLocal_row(rowIn):

            # Grab gransformation matrix
            headTransform_4x4 = np.reshape(rowIn["camera"].values,[4,4])
            headTransform_4x4 = np.array(headTransform_4x4,np.float)

            # Transpose
            headTransform_4x4 = headTransform_4x4.T

            # Inverse
            invHeadTransform_4x4 = np.linalg.inv(headTransform_4x4)

            # Get target loc in world homogenous coordinates
            targetWorld_XYZW = np.hstack([rowIn['targetPos'].values,1]).T

            # Take the dot product
            targetHead_XYZW = np.dot(invHeadTransform_4x4,targetWorld_XYZW)

            # Discard the w term and normalize
            targetHead_XYZ = targetHead_XYZW[0:3] / np.linalg.norm(targetHead_XYZW[0:3])

            # You must return as a list or a tuple
            return {('targeLocal','x'): targetHead_XYZ[0],('targeLocal','y'): targetHead_XYZ[1],('targeLocal','z'): targetHead_XYZ[2]}

        
    oldTargeLocal = sessionDict['processedCalib']['targeLocalPos']
    newTargeLocal = []
    
    for trialRowIdx, trMetaData in sessionDict['trialInfo'].iterrows():
        
        gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])
        trData = gbProcessedCalib_trial.get_group(int(trMetaData['trialNumber']))

        if( str(trMetaData['targetType']) == 'VOR' ):

            # Recalc target position in local coords using the inverse camera matrix
            # RecalcTargetLocal_row returns a list of dictionaries
            
            newTargeLocal = trData.apply(lambda row: recalcTargetLocal_row(row),axis=1)
#             newTargeLocal.append(newTargeLocal)

            # This converts that list of dicts into a multiindexed dataframe
            newTargeLocal = pd.DataFrame.from_records(newTargeLocal.values)
            newTargeLocal.columns = pd.MultiIndex.from_tuples(newTargeLocal.columns)
#             oldTargeLocal.update()

    return newTargeLocal

#     return {('targeLocal','x'): targetHead_XYZ[0],('targeLocal','y'): targetHead_XYZ[1],('targeLocal','z'): targetHead_XYZ[2]}

In [44]:
newTargeLocal = sessionDict['processedCalib']['targeLocalPos']
newTargeLocal.loc[2:3] = [[2,2,2],[3,3,3]]

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/gjdpci/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:

newTargeLocal = recalcTargetLocalPosOnVOR(sessionDict)

newTargeLocal

[]

In [ ]:
tNum = 84 # Trial number

gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])
trData = gbProcessedCalib_trial.get_group(tNum)

trData['newTargeLocalPos']

In [53]:
def recalcTargetLocal_row(rowIn):

        # Grab gransformation matrix
        headTransform_4x4 = np.reshape(rowIn["camera"].values,[4,4])
        headTransform_4x4 = np.array(headTransform_4x4,np.float)

        # Transpose
        headTransform_4x4 = headTransform_4x4.T

        # Inverse
        invHeadTransform_4x4 = np.linalg.inv(headTransform_4x4)

        # Get target loc in world homogenous coordinates
        targetWorld_XYZW = np.hstack([rowIn['targetPos'].values,1]).T

        # Take the dot product
        targetHead_XYZW = np.dot(invHeadTransform_4x4,targetWorld_XYZW)

        # Discard the w term and normalize
        targetHead_XYZ = targetHead_XYZW[0:3] / np.linalg.norm(targetHead_XYZW[0:3])

        # You must return as a list or a tuple
        return {('targeLocal','x'): targetHead_XYZ[0],('targeLocal','y'): targetHead_XYZ[1],('targeLocal','z'): targetHead_XYZ[2]}
    
newTargeLocal = trData.apply(lambda row: recalcTargetLocal_row(row),axis=1)
newTargeLocal

17226    {('targeLocal', 'x'): 0.3652855657874406, ('ta...
17227    {('targeLocal', 'x'): 0.3709782513835461, ('ta...
17228    {('targeLocal', 'x'): 0.37652959160390304, ('t...
17229    {('targeLocal', 'x'): 0.38193652170739106, ('t...
17230    {('targeLocal', 'x'): 0.3848759567338508, ('ta...
                               ...                        
18944    {('targeLocal', 'x'): -0.014123814203053586, (...
18945    {('targeLocal', 'x'): -0.01569967865780512, ('...
18946    {('targeLocal', 'x'): -0.017279415565756594, (...
18947    {('targeLocal', 'x'): -0.018862981779711088, (...
18948    {('targeLocal', 'x'): -0.018862981779711088, (...
Length: 1723, dtype: object

In [54]:
newTargeLocal = pd.DataFrame.from_records(newTargeLocal.values)
newTargeLocal.columns = pd.MultiIndex.from_tuples(newTargeLocal.columns)
newTargeLocal

targeLocal                    
              x         y         z
0      0.365286  0.380514 -0.849574
1      0.370978  0.370799 -0.851401
2      0.376530  0.360992 -0.853177
3      0.381937  0.351098 -0.854901
4      0.384876  0.345695 -0.855784
...         ...       ...       ...
1718  -0.014124  0.826314 -0.563032
1719  -0.015700  0.827005 -0.561975
1720  -0.017279  0.827690 -0.560919
1721  -0.018863  0.828369 -0.559865
1722  -0.018863  0.828369 -0.559865

[1723 rows x 3 columns]